In [40]:
import os, sys, random, operator
import numpy as np

class Environment:
    
    def __init__(self, Ny=5, Nx=5):
        # Define state space
        self.Ny = Ny  # y grid size
        self.Nx = Nx  # x grid size
        self.state_dim = (Ny, Nx)
        # Define action space
        self.action_dim = (4,)  # up, right, down, left
        self.action_dict = {"up": 0, "right": 1, "down": 2, "left": 3}
        self.action_coords = [(-1, 0), (0, 1), (1, 0), (0, -1)]  # translations
        # Define rewards table
        self.R = self._build_rewards()  # R(s,a) agent rewards
        # Check action space consistency
        if len(self.action_dict.keys()) != len(self.action_coords):
            exit("err: inconsistent actions given")

    def reset(self):
        # Reset agent state to top-left grid corner
        self.state = (0, 0)  
        return self.state

    def step(self, action):
        
        
        # Evolve agent state
        state_next = (self.state[0] + self.action_coords[action][0],
                      self.state[1] + self.action_coords[action][1])
        
        if ((state_next[0] == 4) and (state_next[1] == 0)):
            if random.uniform(0, 1) < 0.5:
                state_next = (2,4) 
                
                  
        # Collect reward
        reward = self.R[self.state + (action,)]
          
                   
        # Terminate if we reach bottom-right grid corner
        done = ((state_next[0] == self.Ny - 1) and (state_next[1] == self.Nx - 1)) or ((state_next[0] == self.Ny - 3) and (state_next[1] == self.Nx - 3))
        
        # Update state
        self.state = state_next
        return state_next, reward, done
    
   
    def allowed_actions(self):
        # Generate list of actions allowed depending on agent grid location
        actions_allowed = []
        y, x = self.state[0], self.state[1]
        
        if (x == 1 and y == 0):
            actions_allowed.append(self.action_dict["left"])
            actions_allowed.append(self.action_dict["down"])
           
        elif (x == 2 and y == 1):
            actions_allowed.append(self.action_dict["down"])
            actions_allowed.append(self.action_dict["right"])
            actions_allowed.append(self.action_dict["left"])
            
        elif (x == 3 and y == 0):
            actions_allowed.append(self.action_dict["right"])
            actions_allowed.append(self.action_dict["down"])
            

        else:
            if (y > 0):  # no passing top-boundary
                actions_allowed.append(self.action_dict["up"])
            if (y < self.Ny - 1):  # no passing bottom-boundary
                actions_allowed.append(self.action_dict["down"])
            if (x > 0):  # no passing left-boundary
                actions_allowed.append(self.action_dict["left"])
            if (x < self.Nx - 1):  # no passing right-boundary
                actions_allowed.append(self.action_dict["right"])
        



      
            
        actions_allowed = np.array(actions_allowed, dtype=int)
        return actions_allowed

    def _build_rewards(self):
        # Define agent rewards R[s,a]
        r_goal = 10  # reward for arriving at terminal state (bottom-right corner)
        r_nongoal = -0.1  # penalty for not reaching terminal state
        r_extra = 3
        
        r_sad = -10 
        
        
        R = r_nongoal * np.ones(self.state_dim + self.action_dim, dtype=float)  # R[s,a]
        
        R[2,2] = r_sad
        R[0,3] = r_extra
        
       
        R[self.Ny - 2, self.Nx - 1, self.action_dict["down"]] = r_goal  # arrive from above
        R[self.Ny - 1, self.Nx - 2, self.action_dict["right"]] = r_goal  # arrive from the left
        
        
        return R

class Agent:
    
    def __init__(self, env):
        # Store state and action dimension 
        self.state_dim = env.state_dim
        self.action_dim = env.action_dim
        # Agent learning parameters
        self.epsilon = 0.8  # initial exploration probability
        self.epsilon_decay = 0.99  # epsilon decay after each episode
        self.beta = 0.99  # learning rate
        self.gamma = 0.9  # reward discount factor
        # Initialize Q[s,a] table
        self.Q = np.zeros(self.state_dim + self.action_dim, dtype=float)

    def get_action(self, env):
        # Epsilon-greedy agent policy
        if random.uniform(0, 1) < self.epsilon:
            # explore
            return np.random.choice(env.allowed_actions())
        else:
            # exploit on allowed actions
            state = env.state;
            actions_allowed = env.allowed_actions()
            Q_s = self.Q[state[0], state[1], actions_allowed]
            actions_greedy = actions_allowed[np.flatnonzero(Q_s == np.max(Q_s))]
            return np.random.choice(actions_greedy)

    def train(self, memory):
        # -----------------------------
        # Update:
        #
        # Q[s,a] <- Q[s,a] + beta * (R[s,a] + gamma * max(Q[s,:]) - Q[s,a])
        #
        #  R[s,a] = reward for taking action a from state s
        #  beta = learning rate
        #  gamma = discount factor
        # -----------------------------
        (state, action, state_next, reward, done) = memory
        sa = state + (action,)
        self.Q[sa] += self.beta * (reward + self.gamma*np.max(self.Q[state_next]) - self.Q[sa])

    def display_greedy_policy(self):
        # greedy policy = argmax[a'] Q[s,a']
        greedy_policy = np.zeros((self.state_dim[0], self.state_dim[1]), dtype=int)
        for x in range(self.state_dim[0]):
            for y in range(self.state_dim[1]):
                greedy_policy[y, x] = np.argmax(self.Q[y, x, :])
        print("\nGreedy policy(y, x):")
        print(greedy_policy)
        print()


def main():

    # Settings
    env = Environment(Ny=5, Nx=5) 
    agent = Agent(env)
    
     # Train agent
    print("\nTraining agent...\n")
    N_episodes = 500
    for episode in range(N_episodes):

        # Generate an episode
        iter_episode, reward_episode = 0, 0  
        state = env.reset()  # starting state
        while True:
            action = agent.get_action(env)  # get action
            state_next, reward, done = env.step(action)  # evolve state by action
            agent.train((state, action, state_next, reward, done))  # train agent
            iter_episode += 1
            reward_episode += reward
            
            if iter_episode == 50:
                reward_episode += reward-100
                print('-100 reward')
                break
           
            if done:
                break
            
            
            state = state_next  # transition to next state

        # Decay agent exploration parameter
        agent.epsilon = max(agent.epsilon * agent.epsilon_decay, 0.01)

        # Print
        if (episode == 0) or (episode + 1) % 10 == 0: 
            print("[episode {}/{}] eps = {:.3F} -> iter = {}, rew = {:.1F}".format(
                episode + 1, N_episodes, agent.epsilon, iter_episode, reward_episode))

        # Print greedy policy
        if (episode == N_episodes - 1):
            agent.display_greedy_policy()
            for (key, val) in sorted(env.action_dict.items(), key=operator.itemgetter(1)):
                
                print(" action['{}'] = {}".format(key, val))
            print()

# Driver
if __name__ == '__main__':
    main()



Training agent...

[episode 1/500] eps = 0.792 -> iter = 10, rew = 9.1
-100 reward
[episode 10/500] eps = 0.724 -> iter = 16, rew = 7.7
-100 reward
[episode 20/500] eps = 0.654 -> iter = 28, rew = 6.5
-100 reward
-100 reward
-100 reward
-100 reward
[episode 30/500] eps = 0.592 -> iter = 30, rew = 9.4
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
[episode 40/500] eps = 0.535 -> iter = 44, rew = 35.9
-100 reward
-100 reward
-100 reward
[episode 50/500] eps = 0.484 -> iter = 6, rew = -0.6
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
[episode 60/500] eps = 0.438 -> iter = 50, rew = -55.5
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
[episode 70/500] eps = 0.396 -> iter = 26, rew = 9.8
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
[episode 80/500] eps = 0.358 -> iter = 50, rew = -46.2
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
-100 reward
[episode 90/500] eps = 0.324 -> iter = 50, 